In [48]:
import codecs
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

In [2]:
with codecs.open("./data/encoder_inputs.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    encoder_text = []
    for line in lines:
        data = line.split("\n")[0]
        encoder_text.append(data)

In [3]:
encoder_text

['did you change your hair',
 'i missed you',
 'it was a bratwurst  i was eating lunch',
 'you the new guy',
 "c'mon  i am supposed to give you the tour",
 'north actually  how would you   ',
 'yeah  a couple  we are outnumbered by the cows though',
 'thirtytwo',
 'how many people go here',
 'that i am used to',
 'that girl  i ',
 'who is she',
 'why not',
 'katarina stratford  my my  you have been terrorizing ms blaise again',
 "well yes compared to your other choices of expression this year today's events are quite mild  by the way bobby rictor's gonad retrieval operation went quite well in case you are interested",
 'the point is kat  people perceive you as somewhat ',
 "who's that",
 'that is pat verona the one who was gone for a year i heard he was doing porn movies',
 'he always look so',
 'mandella eat  starving yourself is a very slow way to die',
 'that is this',
 'i realize that the men of this fine institution are severely lacking but killing yourself so you can be with will

In [4]:
with codecs.open("./data/decoder_inputs.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    decoder_text = []
    for line in lines:
        data = line.split("\n")[0]
        decoder_text.append(data)

In [5]:
decoder_text

['<BOS> no <EOS>',
 '<BOS> it says here you exposed yourself to a group of freshmen girls <EOS>',
 '<BOS> with the teeth of your zipper <EOS>',
 '<BOS> so they tell me <EOS>',
 '<BOS> so  which dakota you from <EOS>',
 '<BOS> i was kidding people actually live there <EOS>',
 '<BOS> how many people were in your old school <EOS>',
 '<BOS> get out <EOS>',
 '<BOS> couple thousand most of them evil <EOS>',
 '<BOS> yeah but these guys have never seen a horse  they just jack off to clint eastwood <EOS>',
 '<BOS> you burn you pine you perish <EOS>',
 '<BOS> bianca stratford  sophomore do not even think about it <EOS>',
 '<BOS> i could start with your haircut but it does not matter  she is not allowed to date until her older sister does  and that is an impossibility <EOS>',
 '<BOS> expressing my opinion is not a terrorist action <EOS>',
 '<BOS> i still maintain that he kicked himself in the balls  i was merely a spectator <EOS>',
 '<BOS> tempestuous <EOS>',
 '<BOS> patrick verona   random skid 

In [6]:
# Check dictionary size
full_text = encoder_text + decoder_text

In [7]:
dictionary = []
for text in full_text:
    words = text.split()
    for i in range(0, len(words)):
        if words[i] not in dictionary:
            dictionary.append(words[i])

In [8]:
len(dictionary)

66079

In [9]:
VOCAB_SIZE = 29999
tokenizer = Tokenizer(num_words=VOCAB_SIZE)

In [10]:
tokenizer.fit_on_texts(full_text)
word_index = tokenizer.word_index
len(word_index)

65283

In [11]:
index2word = {}
for k, v in word_index.items():
    if v < (VOCAB_SIZE+1):
        index2word[v] = k
    if v > (VOCAB_SIZE+1):
        continue

In [12]:
index2word

{1: 'bos',
 2: 'eos',
 3: 'you',
 4: 'i',
 5: 'the',
 6: 'to',
 7: 'is',
 8: 'a',
 9: 'not',
 10: 'it',
 11: 'that',
 12: 'do',
 13: 'and',
 14: 'are',
 15: 'of',
 16: 'in',
 17: 'have',
 18: 'what',
 19: 'me',
 20: 'we',
 21: 'he',
 22: 'am',
 23: 'this',
 24: 'for',
 25: 'will',
 26: 'know',
 27: 'was',
 28: 'your',
 29: 'my',
 30: 'on',
 31: 'be',
 32: 'no',
 33: 'with',
 34: 'but',
 35: 'they',
 36: 'would',
 37: 'just',
 38: 'all',
 39: 'like',
 40: 'did',
 41: 'about',
 42: 'get',
 43: 'so',
 44: 'out',
 45: 'if',
 46: 'here',
 47: 'she',
 48: 'him',
 49: 'up',
 50: 'how',
 51: 'got',
 52: 'can',
 53: 'want',
 54: 'think',
 55: 'at',
 56: 'there',
 57: 'one',
 58: 'right',
 59: 'go',
 60: 'now',
 61: 'well',
 62: 'going',
 63: 'her',
 64: 'why',
 65: 'see',
 66: 'as',
 67: 'oh',
 68: 'his',
 69: 'could',
 70: 'yes',
 71: 'who',
 72: 'good',
 73: 'when',
 74: 'cannot',
 75: 'from',
 76: 'where',
 77: 'were',
 78: 'yeah',
 79: 'tell',
 80: 'come',
 81: 'some',
 82: 'been',
 83: 'an

In [13]:
word2index = {}
for k, v in index2word.items():
    word2index[v] = k

In [14]:
word2index

{'bos': 1,
 'eos': 2,
 'you': 3,
 'i': 4,
 'the': 5,
 'to': 6,
 'is': 7,
 'a': 8,
 'not': 9,
 'it': 10,
 'that': 11,
 'do': 12,
 'and': 13,
 'are': 14,
 'of': 15,
 'in': 16,
 'have': 17,
 'what': 18,
 'me': 19,
 'we': 20,
 'he': 21,
 'am': 22,
 'this': 23,
 'for': 24,
 'will': 25,
 'know': 26,
 'was': 27,
 'your': 28,
 'my': 29,
 'on': 30,
 'be': 31,
 'no': 32,
 'with': 33,
 'but': 34,
 'they': 35,
 'would': 36,
 'just': 37,
 'all': 38,
 'like': 39,
 'did': 40,
 'about': 41,
 'get': 42,
 'so': 43,
 'out': 44,
 'if': 45,
 'here': 46,
 'she': 47,
 'him': 48,
 'up': 49,
 'how': 50,
 'got': 51,
 'can': 52,
 'want': 53,
 'think': 54,
 'at': 55,
 'there': 56,
 'one': 57,
 'right': 58,
 'go': 59,
 'now': 60,
 'well': 61,
 'going': 62,
 'her': 63,
 'why': 64,
 'see': 65,
 'as': 66,
 'oh': 67,
 'his': 68,
 'could': 69,
 'yes': 70,
 'who': 71,
 'good': 72,
 'when': 73,
 'cannot': 74,
 'from': 75,
 'where': 76,
 'were': 77,
 'yeah': 78,
 'tell': 79,
 'come': 80,
 'some': 81,
 'been': 82,
 'an': 8

In [15]:
len(word2index) == len(index2word)

True

In [16]:
len(word2index)

29999

In [17]:
encoder_sequences = tokenizer.texts_to_sequences(encoder_text)
decoder_sequences = tokenizer.texts_to_sequences(decoder_text)

In [18]:
encoder_sequences

[[40, 3, 436, 28, 621],
 [4, 950, 3],
 [10, 27, 8, 24746, 4, 27, 1107, 802],
 [3, 5, 186, 168],
 [662, 4, 22, 346, 6, 130, 3, 5, 2407],
 [1211, 388, 50, 36, 3],
 [78, 8, 439, 20, 14, 13613, 115, 5, 5355, 477],
 [7554],
 [50, 274, 116, 59, 46],
 [11, 4, 22, 265, 6],
 [11, 206, 4],
 [71, 7, 47],
 [64, 9],
 [20249, 11389, 29, 29, 3, 17, 82, 17195, 1752, 174],
 [61,
  70,
  3982,
  6,
  28,
  154,
  3146,
  15,
  3809,
  23,
  386,
  3672,
  2758,
  14,
  429,
  9852,
  115,
  5,
  99,
  1202,
  6007,
  1587,
  248,
  429,
  61,
  16,
  327,
  3,
  14,
  668],
 [5, 333, 7, 4340, 116, 20250, 3, 66, 5490],
 [306, 11],
 [11,
  7,
  3248,
  7894,
  5,
  57,
  71,
  27,
  320,
  24,
  8,
  386,
  4,
  254,
  21,
  27,
  128,
  5814,
  979],
 [21, 163, 92, 43],
 [421, 3201, 226, 7, 8, 123, 947, 99, 6, 341],
 [11, 7, 23],
 [4,
  842,
  11,
  5,
  288,
  15,
  23,
  223,
  3606,
  14,
  11390,
  7226,
  34,
  744,
  226,
  43,
  3,
  52,
  31,
  33,
  1759,
  3896,
  7,
  1234,
  5,
  6664,
  15,


In [19]:
decoder_sequences

[[1, 32, 2],
 [1, 10, 278, 46, 3, 4344, 226, 6, 8, 1147, 15, 570, 2],
 [1, 33, 5, 1286, 15, 28, 13725, 2],
 [1, 43, 35, 79, 19, 2],
 [1, 43, 256, 8636, 3, 75, 2],
 [1, 4, 27, 611, 116, 388, 264, 56, 2],
 [1, 50, 274, 116, 77, 16, 28, 178, 317, 2],
 [1, 42, 44, 2],
 [1, 439, 395, 291, 15, 88, 896, 2],
 [1,
  78,
  34,
  145,
  249,
  17,
  96,
  257,
  8,
  1034,
  35,
  37,
  420,
  120,
  6,
  19970,
  19971,
  2],
 [1, 3, 1349, 3, 7772, 3, 12712, 2],
 [1, 6190, 11389, 10576, 12, 9, 135, 54, 41, 10, 2],
 [1,
  4,
  69,
  296,
  33,
  28,
  4900,
  34,
  10,
  100,
  9,
  267,
  47,
  7,
  9,
  1572,
  6,
  686,
  309,
  63,
  1469,
  590,
  100,
  13,
  11,
  7,
  83,
  16810,
  2],
 [1, 11473, 29, 1336, 7, 9, 8, 5942, 1245, 2],
 [1, 4, 146, 4319, 11, 21, 2052, 525, 16, 5, 1561, 4, 27, 3471, 8, 2],
 [1, 2],
 [1, 2250, 7894, 3650, 15267, 2],
 [1, 4, 22, 107, 21, 7, 801, 7644, 15, 128, 129, 11, 792, 2],
 [1, 1840, 2369, 2],
 [1, 37, 8, 103, 2],
 [1, 83, 6460, 6597, 2],
 [1, 34, 770, 5, 

In [20]:
for seqs in decoder_sequences:
    for seq in seqs:
        if seq > 29999:
            print(seq)
            break

In [21]:
VOCAB_SIZE = len(index2word) + 1
VOCAB_SIZE

30000

In [22]:
MAX_LEN = 20
num_samples = len(encoder_sequences)
decoder_output_data = np.zeros((num_samples, MAX_LEN, VOCAB_SIZE), dtype="float32")

In [23]:
encoder_input_data = pad_sequences(encoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')
decoder_input_data = pad_sequences(decoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')

In [24]:
for i, seqs in enumerate(decoder_input_data):
    for j, seq in enumerate(seqs):
        if j > 0:
            decoder_output_data[i][j][seq] = 1.

In [25]:
decoder_output_data.shape

(143864, 20, 30000)

In [40]:
word_embeddings = {}
line_num = 0
with open('./glove_vectors/glove.840B.300d.txt', encoding='utf-8') as glove_file:
    for line in glove_file:
        val = line.split()
        line_num += 1
        try:
            numpy_array = np.asarray(val[1:], dtype='float32')
            word_embeddings[val[0]] = numpy_array
        except:
            print('Ignoring line', line_num)

Ignoring line 52344
Ignoring line 128262
Ignoring line 151103
Ignoring line 200669
Ignoring line 209834
Ignoring line 220780
Ignoring line 253462
Ignoring line 365746
Ignoring line 532049
Ignoring line 717303
Ignoring line 994819
Ignoring line 1123332
Ignoring line 1148410
Ignoring line 1352111
Ignoring line 1499728
Ignoring line 1533810
Ignoring line 1899842
Ignoring line 1921153
Ignoring line 2058967
Ignoring line 2165247


In [41]:
len(word_embeddings)

2195884

In [46]:
word_vec_dimension = 300
def create_word_to_vec(word_vec_dimension, word_index):
    vec_matrix = np.zeros((len(word_index)+1, word_vec_dimension))
    for word, i in word_index.items():
        vec_embedding = word_embeddings.get(word)
        if vec_embedding is not None:
            vec_matrix[i] = vec_embedding
    return vec_matrix

In [47]:
vec_matrix = create_word_to_vec(word_vec_dimension, word2index)

In [49]:
keras_embedding = Embedding(input_dim=VOCAB_SIZE, output_dim=word_vec_dimension, 
                             trainable=True)
keras_embedding.build((None,))
keras_embedding.set_weights([vec_matrix])